## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-11-45-25 +0000,bbc,Taliban official dismisses Trump's hope to 'ta...,https://www.bbc.com/news/articles/c24rz9jeyjjo...
1,2025-09-19-11-39-00 +0000,wsj,"Nvidia, Wayve in Talks Over $500 Million Inves...",https://www.wsj.com/business/autos/nvidia-wayv...
2,2025-09-19-11-38-00 +0000,wsj,The Shifting Ideology Behind Political Violence,https://www.wsj.com/us-news/political-violence...
3,2025-09-19-11-36-33 +0000,nypost,Socialist ‘Squad’ member AOC eyeing 2028 presi...,https://nypost.com/2025/09/19/us-news/rep-alex...
4,2025-09-19-11-30-00 +0000,nypost,St. Anthony’s high school football team turns ...,https://nypost.com/2025/09/19/sports/st-anthon...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,66
56,kimmel,29
55,jimmy,24
139,new,17
369,kirk,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
138,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...,178
298,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,177
12,2025-09-19-11-03-25 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...,148
216,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,146
87,2025-09-19-05-52-01 +0000,nypost,"Seth Meyers calls Trump a ‘visionary, innovato...",https://nypost.com/2025/09/19/entertainment/se...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
138,178,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...
216,80,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
225,64,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...
149,55,2025-09-19-00-00-33 +0000,nypost,Billionaire Mets owner Steve Cohen emerges as ...,https://nypost.com/2025/09/18/us-news/billiona...
236,49,2025-09-18-18-59-08 +0000,bbc,Two shot dead in attack near West Bank-Jordan ...,https://www.bbc.com/news/articles/cr5qv0qmgmjo...
203,43,2025-09-18-20-48-39 +0000,nypost,UK cops show up at Trump-supporting cancer pat...,https://nypost.com/2025/09/18/world-news/uk-co...
295,38,2025-09-18-15-28-22 +0000,nypost,Nvidia makes $5B bet on struggling Intel with ...,https://nypost.com/2025/09/18/business/nvidia-...
336,35,2025-09-18-12-00-00 +0000,wapo,Growing number of Americans say U.S. supports ...,https://www.washingtonpost.com/world/2025/09/1...
279,35,2025-09-18-16-17-43 +0000,latimes,Teenager Celeste Rivas went missing a year ago...,https://www.latimes.com/california/story/2025-...
47,34,2025-09-19-09-30-00 +0000,wsj,The Fed’s rate cut will likely reduce U.S. bor...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
